# Notebook to compute the heat of formation of the experimental reference adsorbates for the CBH approach

created by Bjarne Kreitz (Brown University)

This notebook stores the available literature data and computes the heat of formation of the experimental reference species for the CBH paper


| reaction | $$\Delta H_{rxn}\, /\, \text{kJ mol}^{-1}$$   | temperature / K | Coverage | reference | 
|--- |----------| ------|--------| --------|
| $$O_2 + 2 Ni(111) \rightleftharpoons 2\, O/Ni(111) $$ | -480 | 300 | 1/4 | Wellendorff et al., Surf. Sci. 2015, 640, 36-44|
| $$CO +  Ni(111) \rightleftharpoons \,CO/Ni(111) $$ | -130 | 300 | 1/10 | Wellendorff et al., Surf. Sci. 2015, 640, 36-44|
| $$O_2 + 2 Ni(100) \rightleftharpoons 2\, O/Ni(100) $$ | -530 | 300 | 1/4 | Wellendorff et al., Surf. Sci. 2015, 640, 36-44|
| $$CO +  Ni(100) \rightleftharpoons \,CO/Ni(100) $$ | -126 | 340 | 1/4 | Wellendorff et al., Surf. Sci. 2015, 640, 36-44|
| $$O_2 + 2 Ni(110) \rightleftharpoons 2\, O/Ni(110) $$ | -470 | 300  | 1/4 | Wellendorff et al., Surf. Sci. 2015, 640, 36-44|
| $$CO +  Ni(110) \rightleftharpoons \,CO/Ni(110) $$ | -128 | 380 | 1/4 | Wellendorff et al., Surf. Sci. 2015, 640, 36-44|
| $$O_2 + 2 Ni(211) \rightleftharpoons 2\, O/Ni(211) $$ | -620 | 300 | 1/10 | Wellendorff et al., Surf. Sci. 2015, 640, 36-44|
| $$CO +  Ni(211) \rightleftharpoons \,CO/Ni(211) $$ | -142 | 300 | 1/10 | Wellendorff et al., Surf. Sci. 2015, 640, 36-44|

Values for gas-phase species from the ATcT database (version 1.130)

| species | $$\Delta_f H\, /\, \text{kJ mol}^{-1}$$   | temperature / K |
|--- |----------| ------| 
| $$O_2$$ | 0 | 298.15 | 
| $$CO$$ | -110.520 | 298.15 | 

The experimental heat of formation of the adsorbate is computed at 298K and needs to be corrected to 0K because the heat of reaction are all at 0K.

Correction factors are the following:

| species | $$\Delta\Delta_f H\, /\, \text{kJ mol}^{-1}$$ |
|--- |----------|  
| $$^*O$$ | -1.695 | 
| $$^*CO$$ | 0.05  | 

The heat of formation at 0K is less negative than the heat of formation at 298K from DFT

In [8]:
import numpy as np
import cantera as ct

#import the cti file that contains the NASA polynomials for the gas-phase and surface species

cti_file = 'chem_polynomials.yaml'
    
gas = ct.Solution(cti_file, 'gas')
gas.TPX = 298.15, ct.one_atm, 'N2:1'

#Dictionary with all values from the ATcT database (version 1.130
ATcT={'O2':0, 
      'CO':-110.520
     }

#Dictionary with correction factors to compute the heat of formation at 0K from the heat of formation at 298K
# This correction factor is estimated based on the correction of the DFT heat of formation from 0K to 298K
correction_factor={
      'OX_111':-2.06,
      'OCX_111': 1.32,
      'OX_100':-1.37,
      'OCX_100': 1.9,
      'OX_110':-1.26,
      'OCX_110': 1.54,
      'OX_211':-1.25,
      'OCX_211': 1.78,
        }

"""
We only use the temperature dependence of the heat of formation from the NASA polynomial and not 
the heat of formation, because the ATcT database is continuously updated. This is a function to compute 
the heat of formation at a specific temperature using the ATcT heats of formation and the NASA polynomial.
"""

def get_h_texp(spcs,temp):
    #heat of formation from ATcT database
    hf_atct=ATcT[spcs]
    idx=gas.species_index(spcs)
    
    #Compute the enthalpy of formation at standard conditions from the NASA polynomial
    gas.TP=298.15, ct.one_atm
    hf_std=gas.standard_enthalpies_RT[idx]
    hf_std*=gas.T*ct.gas_constant/1e6 #in kJ/mol
    #print('at std', hf_std)

    #Compute the enthalpy at the temperature of the experiment from the NASA polynomial
    gas.TP=temp, ct.one_atm
    hf=gas.standard_enthalpies_RT[idx]
    hf*=gas.T*ct.gas_constant/1e6 #in kJ/mol
    #print('at exp',hf)
    #Compute the difference in enthalpy between both temperatures from the NASA polynomial
    delta_hf_polynomial=hf-hf_std
    
    #Apply the temperature correction to the heat of formation from the ATcT database
    hf_corrected=hf_atct+delta_hf_polynomial
    return hf_corrected

def get_hf_ads_corr(spcs,temp):

    #Surface species and gas-phase species are in the same phase (called "gas")
    idx=gas.species_index(spcs)
    
    #Compute the enthalpy at standard conditions from the NASA polynomial
    gas.TP=298.15, ct.one_atm
    hf_std=gas.standard_enthalpies_RT[idx]
    hf_std*=gas.T*ct.gas_constant/1e6 #in kJ/mol
    #print('ads at std', hf_std)
    #Compute the enthalpy at the temperature of the experiment from the NASA polynomial
    gas.TP=temp, ct.one_atm
    hf=gas.standard_enthalpies_RT[idx]
    hf*=gas.T*ct.gas_constant/1e6 #in kJ/mol
    #print('ads at exp',hf)
    #Compute the difference in enthalpy from the polynomials, which is used as a correction factor
    delta_hf_polynomial=hf_std-hf
    
    return delta_hf_polynomial


### Determine the heat of formation of O/Ni(111)

This can be directly computed from the reported experimental value. 
$$O_2 + 2 Ni(111) \rightleftharpoons 2\, O/Ni(111) $$ 

$$2\Delta_fH_{O*} - \Delta_fH_{O_2} - 2\Delta_fH_{Ni(111)}=-480 kJ mol^{-1}$$
The heat of reaction for this step is -480 kJ mol$^{-1}$.

The heat of formation of O2 has to be corrected to the experimental temperature of 300K.

In [9]:
hf_hads=-480+get_h_texp('O2', 300)
hf_hads/=2
hf_stp=hf_hads+get_hf_ads_corr('OX_111', 300)
hf_0K=hf_stp-correction_factor['OX_111']

print(f"adsorbate heat of formation at Texp={hf_hads:.4} kJ/mol")
print(f"adsorbate heat of formation at 298 ={hf_stp:.4} kJ/mol")
print(f"adsorbate heat of formation at 0 ={hf_0K:.4} kJ/mol")

adsorbate heat of formation at Texp=-240.0 kJ/mol
adsorbate heat of formation at 298 =-240.0 kJ/mol
adsorbate heat of formation at 0 =-237.9 kJ/mol


### Determine the heat of formation of CO/Ni(111)

This can be directly computed from the reported experimental value. 
$$CO + Ni(111) \rightleftharpoons \,CO/Ni(111) $$ 

$$\Delta_fH_{*CO} - \Delta_fH_{CO} - \Delta_fH_{Ni(111)}=-130 kJ mol^{-1}$$
The heat of reaction for this step is -130 kJ mol$^{-1}$.

The heat of formation of CO has to be corrected to the experimental temperature of 300K.

In [12]:
hf_hads=-120+get_h_texp('CO', 300)
hf_stp=hf_hads+get_hf_ads_corr('OCX_111', 300)
hf_0K=hf_stp-correction_factor['OCX_111']
print(f"adsorbate heat of formation at Texp={hf_hads:.4} kJ/mol")
print(f"adsorbate heat of formation at 298 ={hf_stp:.4} kJ/mol")
print(f"adsorbate heat of formation at 0K ={hf_0K:.4} kJ/mol")

adsorbate heat of formation at Texp=-230.5 kJ/mol
adsorbate heat of formation at 298 =-230.5 kJ/mol
adsorbate heat of formation at 0K =-231.9 kJ/mol


### Determine the heat of formation of O/Ni(100)

This can be directly computed from the reported experimental value. 
$$O_2 + 2 Ni(100) \rightleftharpoons 2\, O/Ni(100) $$ 

$$2\Delta_fH_{O*} - \Delta_fH_{O_2} - 2\Delta_fH_{Ni(100)}=-530 kJ mol^{-1}$$
The heat of reaction for this step is -530 kJ mol$^{-1}$.

The heat of formation of O2 has to be corrected to the experimental temperature of 300K.

In [13]:
hf_hads=-530+get_h_texp('O2', 300)
hf_hads/=2
hf_stp=hf_hads+get_hf_ads_corr('OX_100', 300)
hf_0K=hf_stp-correction_factor['OX_100']

print(f"adsorbate heat of formation at Texp={hf_hads:.4} kJ/mol")
print(f"adsorbate heat of formation at 298 ={hf_stp:.4} kJ/mol")
print(f"adsorbate heat of formation at 0 ={hf_0K:.4} kJ/mol")

adsorbate heat of formation at Texp=-265.0 kJ/mol
adsorbate heat of formation at 298 =-265.0 kJ/mol
adsorbate heat of formation at 0 =-263.6 kJ/mol


### Determine the heat of formation of CO/Ni(100)

This can be directly computed from the reported experimental value. 
$$CO + Ni(100) \rightleftharpoons \,CO/Ni(100) $$ 

$$\Delta_fH_{*CO} - \Delta_fH_{CO} - \Delta_fH_{Ni(100)}=-126 kJ mol^{-1}$$
The heat of reaction for this step is -126 kJ mol$^{-1}$.

The heat of formation of CO has to be corrected to the experimental temperature of 340K.

In [14]:
hf_hads=-126+get_h_texp('CO', 340)
hf_stp=hf_hads+get_hf_ads_corr('OCX_100', 340)
hf_0K=hf_stp-correction_factor['OCX_100']
print(f"adsorbate heat of formation at Texp={hf_hads:.4} kJ/mol")
print(f"adsorbate heat of formation at 298 ={hf_stp:.4} kJ/mol")
print(f"adsorbate heat of formation at 0K ={hf_0K:.4} kJ/mol")

adsorbate heat of formation at Texp=-235.3 kJ/mol
adsorbate heat of formation at 298 =-236.9 kJ/mol
adsorbate heat of formation at 0K =-238.8 kJ/mol


### Determine the heat of formation of O/Ni(110)

This can be directly computed from the reported experimental value. 
$$O_2 + 2 Ni(110) \rightleftharpoons 2\, O/Ni(110) $$ 

$$2\Delta_fH_{O*} - \Delta_fH_{O_2} - 2\Delta_fH_{Ni(110)}=-470 kJ mol^{-1}$$
The heat of reaction for this step is -470 kJ mol$^{-1}$.

The heat of formation of O2 has to be corrected to the experimental temperature of 300K.

In [16]:
hf_hads=-470+get_h_texp('O2', 300)
hf_hads/=2
hf_stp=hf_hads+get_hf_ads_corr('OX_110', 300)
hf_0K=hf_stp-correction_factor['OX_110']

print(f"adsorbate heat of formation at Texp={hf_hads:.4} kJ/mol")
print(f"adsorbate heat of formation at 298 ={hf_stp:.4} kJ/mol")
print(f"adsorbate heat of formation at 0 ={hf_0K:.4} kJ/mol")

adsorbate heat of formation at Texp=-235.0 kJ/mol
adsorbate heat of formation at 298 =-235.0 kJ/mol
adsorbate heat of formation at 0 =-233.7 kJ/mol


### Determine the heat of formation of CO/Ni(110)

This can be directly computed from the reported experimental value. 
$$CO + Ni(110) \rightleftharpoons \,CO/Ni(110) $$ 

$$\Delta_fH_{*CO} - \Delta_fH_{CO} - \Delta_fH_{Ni(110)}=-128 kJ mol^{-1}$$
The heat of reaction for this step is -128 kJ mol$^{-1}$.

The heat of formation of CO has to be corrected to the experimental temperature of 380K.

In [17]:
hf_hads=-128+get_h_texp('CO', 340)
hf_stp=hf_hads+get_hf_ads_corr('OCX_110', 340)
hf_0K=hf_stp-correction_factor['OCX_110']
print(f"adsorbate heat of formation at Texp={hf_hads:.4} kJ/mol")
print(f"adsorbate heat of formation at 298 ={hf_stp:.4} kJ/mol")
print(f"adsorbate heat of formation at 0K ={hf_0K:.4} kJ/mol")

adsorbate heat of formation at Texp=-237.3 kJ/mol
adsorbate heat of formation at 298 =-238.9 kJ/mol
adsorbate heat of formation at 0K =-240.4 kJ/mol


### Determine the heat of formation of O/Ni(211)

This can be directly computed from the reported experimental value. 
$$O_2 + 2 Ni(211) \rightleftharpoons 2\, O/Ni(211) $$ 

$$2\Delta_fH_{O*} - \Delta_fH_{O_2} - 2\Delta_fH_{Ni(211)}=-620 kJ mol^{-1}$$
The heat of reaction for this step is -620 kJ mol$^{-1}$.

The heat of formation of O2 has to be corrected to the experimental temperature of 300K.

In [18]:
hf_hads=-620+get_h_texp('O2', 300)
hf_hads/=2
hf_stp=hf_hads+get_hf_ads_corr('OX_110', 300)
hf_0K=hf_stp-correction_factor['OX_110']

print(f"adsorbate heat of formation at Texp={hf_hads:.4} kJ/mol")
print(f"adsorbate heat of formation at 298 ={hf_stp:.4} kJ/mol")
print(f"adsorbate heat of formation at 0 ={hf_0K:.4} kJ/mol")

adsorbate heat of formation at Texp=-310.0 kJ/mol
adsorbate heat of formation at 298 =-310.0 kJ/mol
adsorbate heat of formation at 0 =-308.7 kJ/mol


### Determine the heat of formation of CO/Ni(211)

This can be directly computed from the reported experimental value. 
$$CO + Ni(211) \rightleftharpoons \,CO/Ni(211) $$ 

$$\Delta_fH_{*CO} - \Delta_fH_{CO} - \Delta_fH_{Ni(211)}=-142 kJ mol^{-1}$$
The heat of reaction for this step is -142 kJ mol$^{-1}$.

The heat of formation of CO has to be corrected to the experimental temperature of 300K.

In [20]:
hf_hads=-142+get_h_texp('CO', 300)
hf_stp=hf_hads+get_hf_ads_corr('OCX_211', 300)
hf_0K=hf_stp-correction_factor['OCX_211']
print(f"adsorbate heat of formation at Texp={hf_hads:.4} kJ/mol")
print(f"adsorbate heat of formation at 298 ={hf_stp:.4} kJ/mol")
print(f"adsorbate heat of formation at 0K ={hf_0K:.4} kJ/mol")

adsorbate heat of formation at Texp=-252.5 kJ/mol
adsorbate heat of formation at 298 =-252.5 kJ/mol
adsorbate heat of formation at 0K =-254.3 kJ/mol
